In [55]:
import json
import pandas as pd
import requests

In [56]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [57]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [58]:
with open("../shared-folder-gald/keys3.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [59]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [60]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [61]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [62]:
access_token = get_access_token(client_key, client_secret)

KeyError: 'access_token'

In [ ]:
liked_videos = []
for username in usernames[5989:6990]:
    liked_videos.append(get_liked_info(username, access_token))

KeyboardInterrupt: 

In [ ]:
combined = {}
for i, username in enumerate(usernames[5989:6990]):
    combined[username] = liked_videos[i]

IndexError: list index out of range

In [ ]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)

            username                   id              creator
0        wickworming  7442599600453668104          diyplantman
1        wickworming  7437410456215571720          diyplantman
2        wickworming  7433277188469804305          diyplantman
3        wickworming  7469245331486608658           hellhungry
4        wickworming  7452933289012104465       triplemcricket
...              ...                  ...                  ...
1611  namaste.to.you  7431902364908473618        beksinthecity
1612  namaste.to.you  7326266144304729387   thehealthyyhabitat
1613  namaste.to.you  7467213209003511086     projectthunderca
1614  namaste.to.you  7466475229917859104  volga_londonstories
1615  namaste.to.you  7348857065122680107       alikaadventure

[1616 rows x 3 columns]


In [ ]:
likes_df.to_csv("liked_videos.csv", mode="a", index=False, header=False)